In [1]:
import os
import sys
PATH="/notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations"
sys.path.append( os.path.join( PATH ) )

from allennlp.predictors.predictor import Predictor
import spacy

BERT_PATH=os.path.join(  PATH, "tests/test_files/models", "bert_model", "bert-base-srl-2019.06.17.tar.gz" )
SPACY_PATH=os.path.join( PATH, "tests/test_files/models", "spacy_model", "spacy-textcat" )

#print( f"loading AllenNLP predictor from {BERT_PATH}" )
#bert_model = Predictor.from_path( BERT_PATH, cuda_device=0 )

#print( f"loading spacy model from {SPACY_PATH}" )
#nlp=spacy.load( SPACY_PATH )

#http://docs.allennlp.org/v0.9.0/api/allennlp.predictors.html
from src.reporting_obligations import ReportingObligationsFinder

reporting_obligations_finder = ReportingObligationsFinder(  BERT_PATH, SPACY_PATH, gpu=0  )


loading AllenNLP predictor from /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/models/bert_model/bert-base-srl-2019.06.17.tar.gz
loading spacy model from /notebook/nas-trainings/arne/DGFISMA/reporting_obligations/code/DGFISMA_reporting_obligations/tests/test_files/models/spacy_model/spacy-textcat


In [2]:
#pip uninstall -y dkpro-cassis==0.5.0

In [3]:
#pip install git+https://github.com/dkpro/dkpro-cassis.git@master

In [4]:
#read in cassis 
#read in the json:
import json
import base64
import re
import sys
import os

from typing import Generator, List

from cassis.typesystem import load_typesystem
from cassis.xmi import load_cas_from_xmi
from cassis import Cas

TEMPLATE_PATH=os.path.join( PATH, "tests/test_files/templates/out.html.template" )


with open( os.path.join( PATH , "tests/test_files/typesystems/typesystem.xml" )  , 'rb') as f:
    typesystem = load_typesystem(f)

#with open( os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/double_nested_list_response.json')) as json_file:
#    response = json.load(json_file)

#with open( os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/small_nested_tables_response.json')) as json_file:
#    response = json.load(json_file)
    
#with open( os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/minus_lesser_of_response.json')) as json_file:
#    response = json.load(json_file)  

#with open( os.path.join( PATH, 'tests/test_files/testing_bug/32011D1208-input.json')) as json_file:
#    response = json.load(json_file)  
    
#with open( os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/32002R0063-input.xmi_outputPar_det.json')) as json_file:
#    response = json.load(json_file)    #busy here
    
#with open( os.path.join( PATH, 'tests/test_files/testing_bug/32014R1141-input.xmi_outputPar_det.json')) as json_file:
#    response = json.load(json_file)   #busy here
    
#with open( os.path.join( PATH, 'tests/test_files/testing_bug/32019L2121-input.xmi_outputPar_det.json')) as json_file:
#    response = json.load(json_file)   #busy here
    
#with open( os.path.join( PATH, 'tests/test_files/testing_bug/31977L0453-input.xmi_outputPar_det.json')) as json_file:
#    response = json.load(json_file)   #busy here
    
#with open( os.path.join( PATH, 'tests/test_files/testing_bug2/32016R1705-input.xmi_output_par_det.json')) as json_file:
#    response = json.load(json_file)

#with open( os.path.join( PATH, 'tests/test_files/testing_bug2/32016R1705-input.xmi_output_RO2.json')) as json_file:
#    response = json.load(json_file)
    
#with open( os.path.join( PATH, 'tests/test_files/testing_bug2/32008D0334-input.xmi_output_Par_det.json')) as json_file:
#    response = json.load(json_file)

#with open( os.path.join( PATH, 'tests/test_files/testing_bug2/32019R0981-input.xmi_output_par_det.json')) as json_file:
#    response = json.load(json_file)

with open( os.path.join( PATH, 'tests/test_files/testing_bug2/32019R0981-input.xmi_definitions.json')) as json_file:
    response = json.load(json_file)
    
#32019L2121-input.xmi_outputRO_gpu_2
    
#with open(  os.path.join( PATH, 'tests/test_files/response_json_paragraph_annotations/doc_bf4ef384-bd7a-51c8-8f7d-d2f61865d767_response.json')) as json_file:
#    response = json.load(json_file)
    
OUTPUT_PATH=os.path.join( PATH, "tests/test_files/output_reporting_obligations/testing_bug3.html" )
    
decoded_cas=base64.b64decode( response[ 'cas_content' ] ).decode( 'utf-8' )

cas=load_cas_from_xmi( decoded_cas, typesystem=typesystem, trusted=True )

In [5]:
from src.transform import ListTransformer

#from src.transform import get_other_lines, transform_lines, flatten_offsets

#from src.utils import SeekableIterator

OldSofaID = 'html2textView'
NewSofaID = 'ListView'
value_between_tagtype = "com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType"
paragraph_type = "de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph"

transformer = ListTransformer( cas )
print( "add list view" )
transformer.add_list_view( OldSofaID )


add list view


In [6]:
print( "get reporting obligations" )
import time
start=time.time()
reporting_obligations_finder.process_sentences( cas, ListSofaID='ListView'  )
reporting_obligations_finder.add_xml_to_cas( cas, TEMPLATE_PATH, ROSofaID='ReportingObligationsView' )
reporting_obligations_finder.print_to_html(  TEMPLATE_PATH, OUTPUT_PATH  )
end=time.time()
print( end-start )

get reporting obligations
loading tokenizer en_core_web_sm
Could not parse "‘5a.   For the purposes of point (i) of point (e) of paragraph 1, the provisions governing the write-down of the nominal or principal amount of the basic own-fund item shall provide for all of the following ❬a) and/or if the trigger event specified in paragraph 8 has occurred in the circumstances described in point (c) of the second subparagraph of that paragraph and a partial write-down would be sufficient to re-establish compliance with the Solvency Capital Requirement, there is a partial write-down of the nominal or principal amount for an amount that is at least sufficient to re-establish compliance with the Solvency Capital Requirement and/or b) and/or if the trigger event specified in paragraph 8 has occurred in the circumstances described in point (c) of the second subparagraph of that paragraph and a partial write-down would not be sufficient to re-establish compliance with the Solvency Capital Requirem

In [10]:
sentence=test_list[1701].split( "|" )[0]
len( sentence.split() )  # skip immediatly!

51949

In [ ]:
#problem with 1701 ?

In [7]:
test_list=cas.get_view( "ListView" ).sofa_string.split( "\n" )
sentence=test_list[1701].split( "|" )[0]

sentence=sentence.rstrip( '\r\n' )
subsentence = re.sub(r'(^[^❮]+|[^❯]+$)',r'', sentence)  #finds everything between " ❮ ❯ " ==>the main sentence
if len(subsentence) > 0: sentence = sentence.replace(subsentence, '', 1)  #remove everything inside " ❮ ❯ " from the string


KeyboardInterrupt: 

In [16]:
' shall ' in subsentence

False

In [7]:
len( cas.get_view( "ListView" ).sofa_string.split( "\n" ) )

1704

NameError: name 'cas' is not defined

In [16]:
tokenized_sentence=reporting_obligations_finder._tokenizer.split_words( test_list[1699 ] )
len( test_list[1699 ].split() )

23538

In [10]:
test_list[ 1699 ]

'Annex XXIII is amended as described in the following list ❮the section ‘Correlation coefficients for earthquake risk in the Hellenic Republic’ is replaced by the following ‖ ‘Correlation coefficients for earthquake risk in the Hellenic Republic ❬j ‖ and/or i ‖ and/or 1 ‖ and/or 2 ‖ and/or 3 ‖ and/or 4 ‖ and/or 5 ‖ and/or 6 ‖ and/or 7 ‖ and/or 8 ‖ and/or 9 ‖ and/or 10 ‖ and/or 11 ‖ and/or 12 ‖ and/or 13 ‖ and/or 14 ‖ and/or 15 ‖ and/or 16 ‖ and/or 17 ‖ and/or 18 ‖ and/or 19 ‖ and/or 20 ‖ and/or 21 ‖ and/or 22 ‖ and/or 23 ‖ and/or 24 ‖ and/or 25 ‖ and/or 1 ‖ and/or 1,00 ‖ and/or 0,75 ‖ and/or 0,75 ‖ and/or 0,75 ‖ and/or 0,75 ‖ and/or 0,75 ‖ and/or 0,75 ‖ and/or 0,75 ‖ and/or 0,75 ‖ and/or 0,75 ‖ and/or 0,25 ‖ and/or 0,25 ‖ and/or 0,00 ‖ and/or 0,25 ‖ and/or 0,00 ‖ and/or 0,25 ‖ and/or 0,25 ‖ and/or 0,00 ‖ and/or 0,00 ‖ and/or 0,00 ‖ and/or 0,00 ‖ and/or 0,00 ‖ and/or 0,50 ‖ and/or 0,25 ‖ and/or 0,25 ‖ and/or 2 ‖ and/or 0,75 ‖ and/or 1,00 ‖ and/or 0,75 ‖ and/or 0,75 ‖ and/or 0,75 ‖ and/o

In [14]:
class Test():
    
    def __init__( self  ):
        pass
                        
    def test_func( self ):
        print( verb )
        
    def run_func( self ):
        for verb in range(10):
            self.test_func()
        

In [15]:
test=Test()
test.run_func()

NameError: name 'verb' is not defined

In [16]:
def test_func(  ):
    print( verb )

verbs=[ 1,2 ]
for verb in verbs:
    test_func()

1
2


In [38]:
#https://github.com/allenai/allennlp/blob/v0.9.0/allennlp/predictors/predictor.py
#https://github.com/allenai/allennlp/blob/v0.9.0/allennlp/predictors/semantic_role_labeler.py

from spacy.tokens import Doc
from allennlp.data.dataset import Batch

from allennlp.nn import util

reporting_obligations_finder._initialize_tokenizer()


input_sentence="This is a test. "*102

tokenized_sentence=reporting_obligations_finder._tokenizer.split_words( input_sentence )
tokenized_sentence=[str( word ) for word in tokenized_sentence]

tokenized_sentence=[ 'on', 'the', 'final', 'system', 'of', 'contributions', 'to', 'the', 'administrative', 'expenditures', 'of', 'the', 'Single', 'Resolution', 'Board']

def valid_length_tokenized_sentence( reporting_obligations_finder: ReportingObligationsFinder, tokenized_sentence: List[str]   ):

    spacy_doc = Doc(reporting_obligations_finder._tokenizer.spacy.vocab, words=tokenized_sentence)
    for pipe in filter(None, reporting_obligations_finder._tokenizer.spacy.pipeline):
        pipe[1](spacy_doc)

    tokens = [token for token in spacy_doc]
    instances = reporting_obligations_finder.bert_model.tokens_to_instances(tokens)

    if not instances:
        return sanitize({"verbs": [], "words": tokens})
    
    #print(instances[0].get_padding_lengths())

    dataset = Batch( instances )
    
    print( instances )
    print( dataset.instances )
    
    dataset.index_instances( reporting_obligations_finder.bert_model._model.vocab  )
    
    #model_input = util.move_to_device(dataset.as_tensor_dict(), 0 )
    
    print ( dataset.as_tensor_dict()['tokens']['tokens'] )
    
    if dataset.as_tensor_dict()['tokens']['tokens'].shape[1] > 512:
        return None
    else:
        return instances


In [ ]:
        tokens = [token for token in spacy_doc]
        instances = self.tokens_to_instances(tokens)

        if not instances:
            return sanitize({"verbs": [], "words": tokens})

In [39]:
import time
start=time.time()
instances=valid_length_tokenized_sentence( reporting_obligations_finder, tokenized_sentence )
end=time.time()
print( end-start )

[]
[]


IndexError: list index out of range

In [ ]:
reporting_obligations_finder.bert_model.predict_instances( instances )

In [ ]:
reporting_obligations_finder.bert_model.predict_

In [40]:
parsed_sentence=reporting_obligations_finder.bert_model.predict_tokenized(
    tokenized_sentence=tokenized_sentence
)

In [41]:
tokenized_sentence

['on',
 'the',
 'final',
 'system',
 'of',
 'contributions',
 'to',
 'the',
 'administrative',
 'expenditures',
 'of',
 'the',
 'Single',
 'Resolution',
 'Board']

In [26]:
parsed_sentence

{'verbs': [],
 'words': ['on',
  'the',
  'final',
  'system',
  'of',
  'contributions',
  'to',
  'the',
  'administrative',
  'expenditures',
  'of',
  'the',
  'Single',
  'Resolution',
  'Board']}

In [4]:
print( cas.get_view( "ListView" ).sofa_string )

23.9.2016|(192, 204)
EN|(255, 257)
Official Journal of the European Union|(308, 346)
L 257/10|(397, 405)
REGULATION (EU) 2016/1705 OF THE EUROPEAN CENTRAL BANK|(465, 519)
of 9 September 2016|(526, 545)
amending Regulation (EC) No 1745/2003 (ECB/2003/9) on the application of minimum reserves (ECB/2016/26)|(552, 655)
THE GOVERNING COUNCIL OF THE EUROPEAN CENTRAL BANK,|(662, 713)
Having regard to the Statute of the European System of Central Banks and of the European Central Bank, and in particular Article 19.1 thereof,|(720, 862)
Having regard to Council Regulation (EC) No 2531/98 of 23 November 1998 concerning the application of minimum reserves by the European Central Bank (1),|(869, 1021)
Whereas, , For the exclusion of interbank liabilities from the reserve base, any standard deduction to be applied to liabilities with a maturity of up to two years within the debt securities category should be based on the euro area-wide macro ratio between: (a) the stock of relevant instruments issu

In [5]:
problematic_sentence=cas.get_view( "ListView" ).sofa_string.split( "\n" )[15]

offset=eval(problematic_sentence.split( "|" )[-1])
assert type( offset ) ==tuple
sentence= "|".join(problematic_sentence.split( "|" )[:-1]) 


sentence=sentence.rstrip( '\r\n' )
subsentence = re.sub(r'(^[^❮]+|[^❯]+$)',r'', sentence)  #finds everything between " ❮ ❯ " ==>the main sentence
if len(subsentence) > 0: sentence = sentence.replace(subsentence, '', 1)  #remove everything inside " ❮ ❯ " from the string


In [6]:
" shall " in subsentence

True

In [7]:
subsentence

'❮Article 3 is amended as follows ❬paragraphs 1 and 2 are replaced by the following ‖ and/or ‘1.   An institution\'s reserve base shall comprise the following liabilities, as defined in the ECB\'s reporting framework laid down in Regulation (EU) No 1071/2013 of the European Central Bank (ECB/2013/33) (*), resulting from the acceptance of funds ⟨deposits ‖ and/or debt securities issued⟩ ‖ and/or If an institution has liabilities in relation to a branch of the same entity, or in relation to the head office or registered office of the same entity, which are located outside participating Member States, it shall include such liabilities in the reserve base ‖ and/or 2.   The following liabilities shall be excluded from the reserve base ⟨liabilities owed to any other institution not included on the list of institutions exempt from the ECB\'s minimum reserve requirements in accordance with Article 2(3) ‖ and/or liabilities owed to the ECB or to a participating NCB⟩ ‖ and/or (*)  Regulation (EU

In [25]:
list_subsentences = re.sub(r' ‖ and/or ', r' and/or ', subsentence).lstrip('❮').rstrip('❯').split(' ‖ ')
list_subsentences[0]

input_sentence=list_subsentences[0][:2300]

reporting_obligations_finder._initialize_tokenizer()

tokenized_sentence=reporting_obligations_finder._tokenizer.split_words( input_sentence )
tokenized_sentence=[str( word ) for word in tokenized_sentence]

#parsed_sentence=reporting_obligations_finder.parse_sentence( tokenized_sentence[:430] )
#parsed_sentence=reporting_obligations_finder.parse_sentence( tokenized_sentence[:200] )  #400 <-> 430


torch.Size([68, 518])


In [19]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM

# OPTIONAL: if you want to have more information on what's happening under the hood, activate the logger as follows
#import logging
#logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize input
text = "Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [30]:
len( tokenizer.tokenize( input_sentence   ) )

510

In [18]:
reporting_obligations_finder.bert_model._model.vocab

Vocabulary with namespaces:  labels, Size: 130 || Non Padded Namespaces: {'*labels', '*tags'}

In [69]:
len( tokenized_sentence )

433

In [ ]:
dataset = Batch(instances)
dataset.index_instances(self.vocab)
model_input = util.move_to_device(dataset.as_tensor_dict(), cuda_device)

In [64]:
len( [ 'This',  'is',  'a', 'test', '.' ]*103 )

515

In [32]:
tokenized_sentence[400:440]

['the',
 'relevant',
 'participating',
 'NCB',
 'of',
 'the',
 'amount',
 'of',
 'liabilities',
 'referred',
 'to',
 'in',
 'points',
 '(',
 'a',
 ')',
 'and',
 '(',
 'b',
 ')',
 'of',
 'paragraph',
 '2',
 ',',
 'the',
 'institution',
 'shall',
 'apply',
 'the',
 'standard',
 'deduction',
 'published',
 'on']

In [25]:
len( tokenized_sentence )

433

In [25]:
input_sentence

'Article 3 is amended as follows ❬paragraphs 1 and 2 are replaced by the following and/or ‘1.   An institution\'s reserve base shall comprise the following liabilities, as defined in the ECB\'s reporting framework laid down in Regulation (EU) No 1071/2013 of the European Central Bank (ECB/2013/33) (*), resulting from the acceptance of funds ⟨deposits and/or debt securities issued⟩ and/or If an institution has liabilities in relation to a branch of the same entity, or in relation to the head office or registered office of the same entity, which are located outside participating Member States, it shall include such liabilities in the reserve base and/or 2.   The following liabilities shall be excluded from the reserve base ⟨liabilities owed to any other institution not included on the list of institutions exempt from the ECB\'s minimum reserve requirements in accordance with Article 2(3) and/or liabilities owed to the ECB or to a participating NCB⟩ and/or (*)  Regulation (EU) No 1071/201

In [8]:
input_sentence=sentence

reporting_obligations_finder._initialize_tokenizer()

tokenized_sentence=reporting_obligations_finder._tokenizer.split_words( input_sentence )
tokenized_sentence=[str( word ) for word in tokenized_sentence]

parsed_sentence=reporting_obligations_finder.parse_sentence( tokenized_sentence )



['Regulation', '(', 'EC', ')', 'No', '1745/2003', '(', 'ECB/2003/9', ')', 'is', 'amended', 'as', 'described', 'in', 'the', 'following', 'list']


In [6]:
#test_string=open( "test.txt" ).read()
#test="\n".join([" ".join(test_string.split( "\n" )[0].split(  )[:490])+"|(10,10)" , " ".join(test_string.split( "\n" )[0].split(  )[:20])+"|(10,10)" ])
#transformer.cas.get_view( "ListView" ).sofa_string=test


In [5]:
print( "get reporting obligations" )
import time
start=time.time()
reporting_obligations_finder.process_sentences( cas, ListSofaID='ListView'  )
reporting_obligations_finder.add_xml_to_cas( cas, TEMPLATE_PATH, ROSofaID='ReportingObligationsView' )
reporting_obligations_finder.print_to_html(  TEMPLATE_PATH, OUTPUT_PATH  )
end=time.time()
print( end-start )

get reporting obligations
loading tokenizer en_core_web_sm
['Whereas', ',', ',', 'For', 'the', 'exclusion', 'of', 'interbank', 'liabilities', 'from', 'the', 'reserve', 'base', ',', 'any', 'standard', 'deduction', 'to', 'be', 'applied', 'to', 'liabilities', 'with', 'a', 'maturity', 'of', 'up', 'to', 'two', 'years', 'within', 'the', 'debt', 'securities', 'category', 'should', 'be', 'based', 'on', 'the', 'euro', 'area', '-', 'wide', 'macro', 'ratio', 'between', ':', '(', 'a', ')', 'the', 'stock', 'of', 'relevant', 'instruments', 'issued', 'by', 'credit', 'institutions', 'and', 'held', 'by', 'other', 'credit', 'institutions', 'and', 'by', 'the', 'ECB', 'and', 'participating', 'national', 'central', 'banks', ';', 'and', '(', 'b', ')', 'the', 'total', 'amount', 'outstanding', 'of', 'such', 'instruments', 'issued', 'by', 'credit', 'institutions', '.', 'The', 'method', 'for', 'applying', 'the', 'standard', 'deduction', 'set', 'out', 'in', 'Article', '3(2', ')', 'of', 'Regulation', '(', 'EC', '

RuntimeError: CUDA error: device-side assert triggered

In [ ]:
reporting_obligations_finder.bert_model.predict( sentence="This is" )

In [ ]:
input_sentence="This is a test sentence."
bert_model = Predictor.from_path( BERT_PATH, cuda_device=1 )
parsed_sentence=bert_model.predict(
    sentence=input_sentence
)

RuntimeError: CUDA error: device-side assert triggered

In [13]:
cas.sofa_string[100:1000]

' <div id="document1" class="tabContent">\n         <div class="tabContent">\n                <div lang="">\n                 \n      <table width="100%" border="0" cellspacing="0" cellpadding="0">\n         <col width="10%">\n         <col width="10%">\n         <col width="60%">\n         <col width="20%">\n         <tbody>\n            <tr>\n               <td>\n                  <p class="hd-date">12.12.2019   </p>\n               </td>\n               <td>\n                  <p class="hd-lg">EN</p>\n               </td>\n               <td>\n                  <p class="hd-ti">Official Journal of the European Union</p>\n               </td>\n               <td>\n                  <p class="hd-oj">L 321/1</p>\n               </td>\n            </tr>\n         </tbody>\n      </table>\n      <hr class="separator">\n      <div>\n         <p class="doc-ti" id="d1e40-1-1">DIRECTIVE (EU) 2019/2121 OF THE EUROPEAN PARL'

In [14]:
cas.get_view( 'html2textView' ).sofa_string[ 100: 1000 ]

'    \n         \n         \n         \n         \n            \n               \n                  12.12.2019   \n               \n               \n                  EN\n               \n               \n                  Official Journal of the European Union\n               \n               \n                  L 321/1\n               \n            \n         \n      \n      \n      \n         DIRECTIVE (EU) 2019/2121 OF THE EUROPEAN PARLIAMENT AND OF THE COUNCIL\n         of 27 November 2019\n         amending Directive (EU) 2017/1132 as regards cross-border conversions, mergers and divisions\n         (Text with EEA relevance)\n         THE EUROPEAN PARLIAMENT AND THE COUNCIL OF THE EUROPEAN UNION,\n         Having regard to the Treaty on the Functioning of the European Union, and in particular Article 50(1) and (2) thereof,\n         Having regard to the proposal from the European Commission,\n         After tran'

NameError: name 'Predictor' is not defined

In [12]:
parsed_sentence

{'verbs': [{'verb': 'is',
   'description': '[ARG1: This] [V: is] [ARG2: a test sentence] .',
   'tags': ['B-ARG1', 'B-V', 'B-ARG2', 'I-ARG2', 'I-ARG2', 'O']}],
 'words': ['This', 'is', 'a', 'test', 'sentence', '.']}

In [1]:
from allennlp.data.tokenizers.word_splitter import SpacyWordSplitter

tokenizer = SpacyWordSplitter(language="en_core_web_sm", pos_tags=True)
tokenized_sentence=tokenizer.split_words( input_sentence )
bert_model.predict_tokenized( [str( word ) for word in tokenized_sentence] )

NameError: name 'input_sentence' is not defined

In [25]:
len( tokenized_sentence )

6

In [19]:
type( tokenized_sentence[0] )

allennlp.data.tokenizers.token.Token

In [20]:
str( tokenized_sentence[0] )

'This'

In [39]:
from allennlp.data.tokenizers.word_splitter import SpacyWordSplitter

tokenizer = SpacyWordSplitter(language="en_core_web_sm", pos_tags=True) 
len(tokenizer.split_words( test.split( "\n" )[0] ))

532

In [48]:
len( test_string )

2156

In [66]:
if (len( "")  > 2 or len( "test test test test".split()) > 2) :
    print( "sd" ) 

sd


In [51]:
len( test_string.split( ) )

332

In [19]:

len( tokenizer.split_words( test_string ) )

# self._tokenizer = SpacyWordSplitter(language='en_core_web_sm', pos_tags=True) 


2551

In [12]:
import time
start=time.time()
try: 
    parsed_sentence=bert_model.predict(sentence=test[2] )
except RuntimeError:
    print( "RunTimeError, could not parse " )
end=time.time()
print( end-start )

4.5183799266815186


In [13]:
RuntimeError

RuntimeError

In [47]:
def break_long_paragraphs(text: str ) -> list:
    
    if len(text) > 500:
        text = re.sub(r'([^❮❬‖❭❯]{500}(?:[^❮❬‖❭❯.]|[.][^❮❬‖❭❯ ])*[.]) (?=[A-Z])', r'\1\n', text, re.MULTILINE + re.UNICODE)
    
    break_lines = [line.strip() for line in text.split( "\n" ) if line.strip()]

    return break_lines

In [48]:
import re
broken=break_long_paragraphs( test[2] )

In [53]:
broken[0]

"As a key element of possessing European legal status, European political parties and European political foundations should have European legal personality. The acquisition of European legal personality should be subjected to requirements and procedures to protect the interests of the Member State of the seat, of the applicant for European legal status ('the applicant') and of any third parties concerned. In particular, any pre-existing national legal personality should be converted into a European legal personality and any individual rights and obligations that have accrued to the former national legal entity should be transferred to the new European legal entity."

In [46]:
import time
start=time.time()
bert_model.predict( sentence=test[2] )
end=time.time()
print( end-start )

4.548712491989136


In [34]:
test[2][:700]

"As a key element of possessing European legal status, European political parties and European political foundations should have European legal personality. The acquisition of European legal personality should be subjected to requirements and procedures to protect the interests of the Member State of the seat, of the applicant for European legal status ('the applicant') and of any third parties concerned. In particular, any pre-existing national legal personality should be converted into a European legal personality and any individual rights and obligations that have accrued to the former national legal entity should be transferred to the new European legal entity. Moreover, in order to facil"

In [23]:
test[2][:500]

"As a key element of possessing European legal status, European political parties and European political foundations should have European legal personality. The acquisition of European legal personality should be subjected to requirements and procedures to protect the interests of the Member State of the seat, of the applicant for European legal status ('the applicant') and of any third parties concerned. In particular, any pre-existing national legal personality should be converted into a Europe"

In [77]:
test=open( "test.txt" ).read().split( "\n" )
test[1]

'test test and more. And more ❮ And then. And more and then. And more ❯ And more and then. And'

In [78]:
break_long_paragraphs( test[1] )

['test test and more.',
 'And more ❮ And then. And more and then.',
 'And more ❯ And more and then.',
 'And']

In [56]:
test

['test test and more. And more ❮ And then. And more and then. And more❯ And more and then. And']

In [42]:
test

['The termination of European legal personality should be subject to requirements and procedures to protect the interests of the Union, of the Member State of the seat, of the European political party or European political foundation and of any third parties concerned. In particular, if the European political party or European political foundation acquires legal personality under the law of the Member State of its seat, this should be considered as a conversion of the European legal personality and any individual rights and obligations that the former European legal entity has respectively acquired or incurred should be transferred to the national legal entity. Moreover, in order to facilitate continuity of activity, safeguards should be put in place to prevent the Member State concerned from applying prohibitive conditions to such conversion. If the European political party or European political foundation does not acquire legal personality in the Member State of its seat, it should b

In [ ]:
print( "get reporting obligations" )
import time
start=time.time()
reporting_obligations_finder = ReportingObligationsFinder( cas, bert_model, nlp )
reporting_obligations_finder.process_sentences( ListSofaID='ListView'  )
reporting_obligations_finder.add_xml_to_cas( TEMPLATE_PATH, ROSofaID='ReportingObligationsView' )
reporting_obligations_finder.print_to_html(  TEMPLATE_PATH, OUTPUT_PATH  )
end=time.time()
print( end-start )

In [60]:
def break_long_paragraphs(text: str ) -> list:
    
    if len(text) > 10:
        text = re.sub(r'([^❮❬‖❭❯]{10}(?:[^❮❬‖❭❯.]|[.][^❮❬‖❭❯ ])*[.]) (?=[A-Z])', r'\1\n', text, re.MULTILINE + re.UNICODE)
    
    break_lines = [line.strip() for line in text.split( "\n" ) if line.strip()]

    return break_lines

In [64]:
text= "As a key element of possessing European legal status, European political parties and European political foundations should have European legal personality.  ❮ Test and.  ❯ ❯ . The acquisition of European legal personality should be subject to requirements and procedures to protect the interests of the Member State of the seat, of the applicant for European legal status (the applicant) and of any third parties concerned. In particular, any pre-existing national legal personality should be converted into a European legal personality and any individual rights and obligations that have accrued to the former national legal entity should be transferred to the new European legal entity. Moreover, in order to facilitate continuity of activity, safeguards should be put in place to prevent the Member State concerned from applying prohibitive conditions to such conversion. The Member State of the seat should be able to specify which types of national legal persons may be converted into European legal persons, and to withhold its agreement to the acquisition of European legal personality under this Regulation until adequate guarantees are provided, in particular, for the legality of the applicant statutes under the laws of that Member State or for the protection of creditors or holders of other rights in respect of any pre-existing national legal personality."

In [65]:
len( text )

1371

In [66]:
break_long_paragraphs(text)

['As a key element of possessing European legal status, European political parties and European political foundations ❮ should have European legal personality.  ❮ Test and.  ❯ ❯ . The acquisition of European legal personality should be subject to requirements and procedures to protect the interests of the Member State of the seat, of the applicant for European legal status (the applicant) and of any third parties concerned.',
 'In particular, any pre-existing national legal personality should be converted into a European legal personality and any individual rights and obligations that have accrued to the former national legal entity should be transferred to the new European legal entity.',
 'Moreover, in order to facilitate continuity of activity, safeguards should be put in place to prevent the Member State concerned from applying prohibitive conditions to such conversion.',
 'The Member State of the seat should be able to specify which types of national legal persons may be converted

In [30]:
print( "get reporting obligations" )
import time
start=time.time()
reporting_obligations_finder = ReportingObligationsFinder( cas, bert_model, nlp )
reporting_obligations_finder.process_sentences( ListSofaID='ListView'  )
reporting_obligations_finder.add_xml_to_cas( TEMPLATE_PATH, ROSofaID='ReportingObligationsView' )
reporting_obligations_finder.print_to_html(  TEMPLATE_PATH, OUTPUT_PATH  )
end=time.time()
print( end-start )

get reporting obligations
processing sentence: 5. In the case of a sample, only a selection of the credit institutions and other institutions in the potential reporting population shall be asked to report. The variables that shall be estimated by means of the sample are the interest rates and the amounts of new business and the interest rates on outstanding amounts. They are referred to as sampling variables. In order to minimise the risk that the results of a sample survey deviate from the true (unknown) values in the potential reporting population, the sample shall be constructed in such a way that it is representative of the potential reporting population. For the purpose of MFI interest rate statistics a sample shall be considered representative if all the characteristics that are relevant for MFI interest rate statistics and inherent in the potential reporting population are also reflected in the sample. For drawing the initial sample, NCBs may use suitable proxies and models to p

RuntimeError: index out of range: Tried to access index 512 out of table with 511 rows. at /opt/conda/conda-bld/pytorch_1579022030672/work/aten/src/TH/generic/THTensorEvenMoreMath.cpp:418

In [ ]:
3.8721108436584473

In [34]:
len( (test_text*3).split() )

541

In [35]:
test_text

'5. In the case of a sample, only a selection of the credit institutions and other institutions in the potential reporting population shall be asked to report. The variables that shall be estimated by means of the sample are the interest rates and the amounts of new business and the interest rates on outstanding amounts. They are referred to as sampling variables. In order to minimise the risk that the results of a sample survey deviate from the true (unknown) values in the potential reporting population, the sample shall be constructed in such a way that it is representative of the potential reporting population. For the purpose of MFI interest rate statistics a sample shall be considered representative if all the characteristics that are relevant for MFI interest rate statistics and inherent in the potential reporting population are also reflected in the sample. For drawing the initial sample, NCBs may use suitable proxies and models to produce the sampling scheme even if the underly

In [36]:
len(test_text)

1124

In [42]:
text="2.   The data shall contain at least the following elements ❮the identity and contact details of each contribution debtor as determined in accordance with Article 4 of Regulation (EU) No 1163/2014 for the purpose of the supervisory fees ‖ the fee factors determined in accordance with Article 10 of Regulation (EU) No 1163/2014 ‖ whether a contribution debtor is significant in accordance with Article 6(4) of Regulation (EU) No 1024/2013 or is an entity or group in relation to which the ECB has decided in accordance with Article 6(5)(b) of Regulation (EU) No 1024/2013 to exercise directly all of the relevant powers ‖ any data that the ECB has determined in the absence of reporting from a contribution debtor, in accordance with Article 10(5) of Regulation (EU) No 1163/2014 ‖ the validity date underlying the fee calculation of each contribution debtor determining the duration the contribution debtor was subject to the supervisory fee and any change of status in accordance with Article 7(2) of Regulation (EU) No 1163/2014 in the given fee period❯"

text = re.sub(r'([^❮❬‖❭❯]{10}(?:[^❮❬‖❭❯.]|[.][^❮❬‖❭❯ ])*[.]) (?=[A-Z])', r'\1\n', text, re.MULTILINE + re.UNICODE)


In [47]:
text.split( "\n" )

['2.   The data shall contain at least the following elements ❮the identity and contact details of each contribution debtor as determined in accordance with Article 4 of Regulation (EU) No 1163/2014 for the purpose of the supervisory fees ‖ the fee factors determined in accordance with Article 10 of Regulation (EU) No 1163/2014 ‖ whether a contribution debtor is significant in accordance with Article 6(4) of Regulation (EU) No 1024/2013 or is an entity or group in relation to which the ECB has decided in accordance with Article 6(5)(b) of Regulation (EU) No 1024/2013 to exercise directly all of the relevant powers ‖ any data that the ECB has determined in the absence of reporting from a contribution debtor, in accordance with Article 10(5) of Regulation (EU) No 1163/2014 ‖ the validity date underlying the fee calculation of each contribution debtor determining the duration the contribution debtor was subject to the supervisory fee and any change of status in accordance with Article 7(2

In [ ]:
#check performance issue...

In [46]:
'❮'=='❮'

True

In [56]:
def print_break_long_paragraphs(text):
    if len(text) > 10:
        text = re.sub(r'([^❮❬‖❭❯]{10}(?:[^❮❬‖❭❯.]|[.][^❮❬‖❭❯ ])*[.]) (?=[A-Z])', r'\1\n', text, re.MULTILINE + re.UNICODE)
    print(text)

In [57]:
print_break_long_paragraphs( text  )

2.   The data shall contain at least the following elements ❮the identity and contact details of each contribution debtor as determined in accordance with Article 4 of Regulation (EU) No 1163/2014 for the purpose of the supervisory fees ‖ the fee factors determined in accordance with Article 10 of Regulation (EU) No 1163/2014 ‖ whether a contribution debtor is significant in accordance with Article 6(4) of Regulation (EU) No 1024/2013 or is an entity or group in relation to which the ECB has decided in accordance with Article 6(5)(b) of Regulation (EU) No 1024/2013 to exercise directly all of the relevant powers ‖ any data that the ECB has determined in the absence of reporting from a contribution debtor, in accordance with Article 10(5) of Regulation (EU) No 1163/2014 ‖ the validity date underlying the fee calculation of each contribution debtor determining the duration the contribution debtor was subject to the supervisory fee and any change of status in accordance with Article 7(2) 

In [2]:
from src.transform import ListTransformer
from src.reporting_obligations import ReportingObligationsFinder

from src.transform import get_other_lines, transform_lines, flatten_offsets, postprocess_nested_lines

from src.utils import SeekableIterator

#print( f"loading AllenNLP predictor from {BERT_PATH}" )
#bert_model = Predictor.from_path( BERT_PATH )

#print( f"loading spacy model from {SPACY_PATH}" )
#nlp=spacy.load( SPACY_PATH )

OldSofaID = 'html2textView'
NewSofaID = 'ListView'
value_between_tagtype = "com.crosslang.uimahtmltotext.uima.type.ValueBetweenTagType"
paragraph_type = "de.tudarmstadt.ukp.dkpro.core.api.segmentation.type.Paragraph"

value_between_tagtype_generator=cas.get_view( OldSofaID ).select( value_between_tagtype )        

#value_between_tagtype_generator=get_deepest_child_tags(  cas, OldSofaID , \
#                   value_between_tagtype=value_between_tagtype  )

seek_vbtt=SeekableIterator( iter(value_between_tagtype_generator) )

lines, offsets=get_other_lines( cas , OldSofaID, seek_vbtt, 'root', paragraph_type=paragraph_type )

#flatten_offsets( offsets )

#lines, offsets=postprocess_nested_lines( lines, offsets  )